In [1]:
# why classification
    # 推荐里面内容安全，我们需要对我们的内容进行判别，标题党识别，黄反内容识别，低俗识别，不相关
    # 推荐里面画像类，我们需要对我们的内容进行分类， 一级类目（历史、人文、咨询、电商。。。。。）
    # 商品的多级类目：蔬菜-->根茎类蔬菜-->萝卜[白萝卜、红萝卜、胡萝卜]
    # 推荐里面评论挖掘，我们需要对我们的评论去做情感分析，将positive尽量的排在negative的前面
    # 搜索系统，一个query，它是多个类别的刻画，小猪佩奇，是一个动画片，电影，我们需要进行意图识别，将多意图的内容去做多样化
    # 推荐系统里面去做内容维度的打散，也是需要类别信息
    # 推荐系统里面去做内容维度的频控，也是需要类别信息

In [2]:
# task
    # input： 我们输入的是一串文本内容的描述信息（一个一个的文本），模型需要能够接受上述的文本的描述信息
    # output：判定这一段文本内容的类别[预定义的类]

In [3]:
# 之前，我们没有学习embedding技术，我们采用的有可能是one hot方式去描述文本，但是这种方式不具有一定的泛化能力，高度稀疏、歧义问题无法解决，无语义信息。
# 我们引入了embedding技术，我们的embedding当前已知的是word embedding，我们输入的序列信息是一串的word序列，那么我们怎么去通过这一串序列
# 描述我们的文本内容呢？
    # 我们可以去把当前这段文本里面所有的词的embedding相加起来，作为我们整个句子的embedding描述（可以是加、也可以是平均、也可以使按位取最大）
    # 当前embedding作为分类模型的输入（LR、XGB、LGB、SVM）。

In [4]:
# word2vec+LR、fasttext、textCNN、lstm + Attention、bert等

In [5]:
# 我们采用word2vec训练得到word的embedding，能够唯一标识一个词的向量，然后通过词向量得到句子向量，我们就可以把句子向量作为特征
# 通过特征进行学习，就能简单的去做一个文本分类模型，这个是最简单的一种方式，但是准确率也还可以
# 训练集共有24000条样本，12个分类，每个分类2000条样本。
# 测试集共有12000条样本，12个分类，每个分类1000条样本。

In [6]:
# 之前介绍过pandas，此处不做赘述，使用pandas库的read_csv方法加载文本文件

In [7]:
import pandas as pd
# read_csv方法中有3个参数，第1个参数是加载文本文件的路径，第2个关键字参数sep是分隔符，第3个关键字参数header是文本文件的第1行是否为字段名
train_df = pd.read_csv('data/text_train.txt', sep='\t', header=None)
train_df.columns = ['分类', '内容']
train_df.head() # 打印训练集前5行

分类                                                 内容
0  娱乐  《青蛇》造型师默认新《红楼梦》额妆抄袭（图） 凡是看过电影《青蛇》的人，都不会忘记青白二蛇的...
1  娱乐  ６．１６日剧榜　＜最后的朋友＞　亮最后杀招成功登顶 《最后的朋友》本周的电视剧排行榜单依然只...
2  娱乐  超乎想象的好看《纳尼亚传奇２：凯斯宾王子》 现时资讯如此发达，搜狐电影评审团几乎人人在没有看...
3  娱乐  吴宇森：赤壁大战不会出现在上集 “希望《赤壁》能给你们不一样的感觉。”对于自己刚刚拍完的影片...
4  娱乐  组图：《多情女人痴情男》陈浩民现场耍宝 陈浩民：外面的朋友大家好，现在是搜狐现场直播，欢迎《...

In [8]:
# 查看训练集每个分类的名字以及样本数量
for name, group in train_df.groupby(train_df.columns[0]):
    print(name,len(group))

体育 2000
健康 2000
女人 2000
娱乐 2000
房地产 2000
教育 2000
文化 2000
新闻 2000
旅游 2000
汽车 2000
科技 2000
财经 2000


In [9]:
# 加载测试集并查看每个分类的名字以及样本数量
test_df = pd.read_csv('data/text_test.txt', sep='\t', header=None)
for name, group in test_df.groupby(test_df.columns[0]):
    print(name, len(group))

体育 1000
健康 1000
女人 1000
娱乐 1000
房地产 1000
教育 1000
文化 1000
新闻 1000
旅游 1000
汽车 1000
科技 1000
财经 1000


In [10]:
# 文本分类不是任何词都是重要的判断语句，针对一些没有任何代表意义的词（的、了、么、吗）应该去掉，这种词容易对模型进行干扰
with open('data/stopwords.txt', encoding='utf8') as file:
    line_list = file.readlines()
    stopword_list = [k.strip() for k in line_list]
    stopword_set = set(stopword_list)
    print('停顿词列表，即变量stopword_list中共有%d个元素' %len(stopword_list))
    print('停顿词集合，即变量stopword_set中共有%d个元素' %len(stopword_set))

停顿词列表，即变量stopword_list中共有1452个元素
停顿词集合，即变量stopword_set中共有1213个元素


In [11]:
# 对训练集的24000条样本循环遍历，使用jieba库的cut方法获得分词列表赋值给变量cutWords。
# 判断分词是否为停顿词，如果不为停顿词，则添加进变量cutWords中
import jieba
import time

In [12]:
cutWords_list = []
startTime = time.time()
# train_df，所有训练样本
content_series = train_df['内容']
for i in range(len(content_series)):
    # 取第i行的content
    content = content_series.iloc[i]
    # Model type. True for full pattern, False for accurate pattern
    # 原句：我来到北京清华大学　True结果：我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
    cutWords = [k for k in jieba.cut(content, True) if k not in stopword_set]
    if (i+1) % 3000 == 0:
        usedTime = time.time() - startTime
        print('前%d篇文章分词共花费%.2f秒' %(i+1, usedTime))
    cutWords_list.append(cutWords)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\longx\AppData\Local\Temp\jieba.cache
Loading model cost 0.984 seconds.
Prefix dict has been built successfully.


前3000篇文章分词共花费26.40秒
前6000篇文章分词共花费63.54秒
前9000篇文章分词共花费76.60秒
前12000篇文章分词共花费94.57秒
前15000篇文章分词共花费108.26秒
前18000篇文章分词共花费123.84秒
前21000篇文章分词共花费151.77秒
前24000篇文章分词共花费164.37秒


In [13]:
# 从上面的运行结果可以看出，对24000篇文章进行分词共使用164.37秒，即2分44.37秒
# 将分词结果保存为文本文件data/txt_cutWords_list.txt
txtFilePath = 'data/txt_cutWords_list.txt'
with open(txtFilePath, 'w', encoding='utf8') as file:
    for cutWords in cutWords_list:
        file.write(' '.join(cutWords))
        file.write('\n')

In [14]:
# 载入分词文本文件，200M
with open(txtFilePath, 'r', encoding='utf8') as file:
    cutWords_list = [k.split(' ') for k in file.readlines()]

In [15]:
from gensim.models import Word2Vec

In [17]:
startTime = time.time()
# sg如果是0，则是CBOW模型；是1则是Skip-Gram模型；默认是0即CBOW模型。
# hs如果是0，则是Negative Sampling；是1的话并且负采样个数negative大于0，则是Hierarchical Softmax。默认是0即Negative Sampling。
# 即使用Negative Sampling时负采样的个数，默认是5。推荐在[3,10]之间。
word2vec_model = Word2Vec(cutWords_list, sg=1, window=5, min_count=10, sample=0.001, negative=3, hs=1, workers=4)
usedTime = time.time() - startTime
print('形成word2vec模型共花费%.2f秒' %usedTime)

形成word2vec模型共花费1006.41秒


In [16]:
# 形成word2vec模型共花费1006.41秒
# 1006.41秒 ≈17分钟

In [17]:
# 模型保存
# 1.保存模型&模型结构，结果支持增量训练
# 2.只保存模型结果，不支持增量训练

In [18]:
# 先来看支持增量训练的model

In [19]:
# 1.模型&结构
word2vec_model.save("models/w2v_1015.m")

In [20]:
# 加载模型
word2vec_model = Word2Vec.load("models/w2v_1015.m")

In [21]:
#模型调用
vector = word2vec_model.wv['电视']  # get numpy vector of a word
vector

array([ 0.22544637,  0.05437879,  0.47380644,  0.4441837 , -0.33141288,
       -0.04201641,  0.01990791,  0.59349465, -0.03621645, -0.14820674,
       -0.11806272, -0.34715977,  0.2559092 ,  0.09826874, -0.10218447,
       -0.0340074 ,  0.05200435, -0.41739056, -0.13428901, -0.15272635,
        0.6006828 , -0.07389024, -0.6695627 , -0.57157135, -0.11465421,
       -0.28795993, -0.22859003,  0.35007384,  0.0403269 , -0.03749996,
       -0.4303904 , -0.02383559,  0.23838435,  0.03355436, -0.07165477,
       -0.2616719 , -0.01251765, -0.10816049,  0.00994392,  0.08617909,
       -0.23027326,  0.00509146, -0.15504427, -0.27670372,  0.29238847,
       -0.1469976 , -0.43877786,  0.36644772,  0.00733865,  0.6069893 ,
        0.26095992, -0.50798166, -0.1830667 , -0.19897059,  0.34146804,
       -0.35952276,  0.56561244, -0.13904573,  0.35155538, -0.13257404,
       -0.1967044 , -0.65154815, -0.30967227,  0.19664688, -0.0404706 ,
        0.05540451,  0.16848293, -0.45153904,  0.23347214, -0.27

In [22]:
sims =word2vec_model.wv.most_similar("电视剧",topn=10)  #get other similar words
sims

[('电视', 0.8247318267822266),
 ('剧', 0.7823254466056824),
 ('热播', 0.7768886089324951),
 ('电视剧集', 0.7714131474494934),
 ('剧里', 0.7667648196220398),
 ('剧集', 0.7212305665016174),
 ('电视台', 0.6916954517364502),
 ('台湾电视', 0.688531219959259),
 ('新剧', 0.6872106790542603),
 ('荧屏', 0.6865444779396057)]

In [23]:
# 增量训练模型
word2vec_model.train(["最后","朋友","本周","电视","电视剧","排行","排行榜","榜单","依然","九部","作品","收视","收视率","超过","１０％"], total_examples=1, epochs=1)

(29, 34)

In [24]:
# 非增量训练模式【保存为二进制embedding】

In [25]:
word2vec_model.wv.save_word2vec_format("models/w2v_model.bin",binary=True)

In [26]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('models/w2v_model.bin',binary=True)

In [27]:
# 二进制embedding，查看与摄影含义最相近的词
model.most_similar('电视剧')

[('电视', 0.8247318267822266),
 ('剧', 0.7807431221008301),
 ('热播', 0.7768886089324951),
 ('电视剧集', 0.7714131474494934),
 ('剧里', 0.7667648196220398),
 ('剧集', 0.7212305665016174),
 ('电视台', 0.6916954517364502),
 ('台湾电视', 0.688531219959259),
 ('新剧', 0.6872106790542603),
 ('荧屏', 0.6865444779396057)]

In [28]:
# embedding计算

In [29]:
# 查看女人+先生-男人的结果【非二进制model】
word2vec_model.wv.most_similar(positive=['女人', '先生'], negative=['男人'], topn=1)

[('女士', 0.7417411208152771)]

In [30]:
# 查看女人+先生-男人的结果【二进制model】
model.most_similar(positive=['女人', '先生'], negative=['男人'], topn=1)

[('女士', 0.7417411208152771)]

In [31]:
# 对于每一篇文章，获取文章的每一个分词在word2vec模型的相关词向量。
# 然后把一篇文章的所有分词在word2vec模型中的相关性向量求和取平均数，即此篇文章在word2vec模型中的相关文档向量

In [32]:
import numpy as np
def get_contentVector(cutWords, word2vec_model):
    # 变量vector_list是通过列表推导式得出单篇文章所有分词的词向量，通过np.array方法转成ndarray对象再对每一列求平均值
    vector_list = [word2vec_model.wv[k] for k in cutWords if k in word2vec_model.wv]
    contentVector = np.array(vector_list).mean(axis=0)
    return contentVector

In [ ]:
# 不是加起来就可以代表了吗？再取平均的目的是？
# 这里我可以用任何策略，只要是最终效果好就行了。mean只是其中一种策略，大家可以自己尝试直接求和，也可以尝试按位取最大，按位取最小等。
# A [0.01, 0.13, -0.23]
# B [-0.29, 0.28, 0.06]
# A ^ B = [0.01, 0.28, 0.06]
# AB = [0.01, 0.13, -0.23, 0.01, 0.28, 0.06]

In [33]:
# 训练样本的enbedding
startTime = time.time()
# content的embedding list
contentVector_list = []
for i in range(len(cutWords_list)):
    # 每一篇文章
    cutWords = cutWords_list[i]
    if (i+1) % 3000 == 0:
        usedTime = time.time() - startTime
        print('前%d篇文章内容表示成向量共花费%.2f秒' %(i+1, usedTime))
    contentVector_list.append(get_contentVector(cutWords, word2vec_model))
X = np.array(contentVector_list)

前3000篇文章内容表示成向量共花费17.89秒
前6000篇文章内容表示成向量共花费45.27秒
前9000篇文章内容表示成向量共花费56.99秒
前12000篇文章内容表示成向量共花费70.53秒
前15000篇文章内容表示成向量共花费80.23秒
前18000篇文章内容表示成向量共花费91.24秒
前21000篇文章内容表示成向量共花费111.13秒
前24000篇文章内容表示成向量共花费119.35秒


In [34]:
# 因为形成特征矩阵的花费时间较长，为了避免以后重复花费时间，把特征矩阵保存为文件。使用ndarray对象的dump方法，
# 需要1个参数，数据类型为字符串，为保存文件的文件名
txtFilePath = 'data/X.txt'
X.dump(txtFilePath)

In [35]:
X.shape

(24000, 100)

In [36]:
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()
y = labelEncoder.fit_transform(train_df['分类'])

In [37]:
y.shape

(24000,)

In [38]:
# 方法1，直接训练model
from sklearn.linear_model import LogisticRegression
# 对训练样本进行分割，3:7,2:8
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
logisticRegression_model = LogisticRegression()
# 80%样本
logisticRegression_model.fit(train_X, train_y)
# 20%样本
logisticRegression_model.score(test_X, test_y)

D:\softWare\anacondas\envs\py365\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7860416666666666

In [39]:
# 为了避免1111，...,0000这种情况

In [40]:
# 方法2，融入shuffle
from sklearn.linear_model import LogisticRegression
# 重新洗牌和分裂迭代次数，为cross_val_score迭代器自定义其他交叉验证策略
from sklearn.model_selection import ShuffleSplit
# 交叉验证（KFold）
from sklearn.model_selection import cross_val_score

cv_split = ShuffleSplit(n_splits=5, train_size=0.8, test_size=0.2)
score_ndarray = cross_val_score(LogisticRegression(), X, y, cv=cv_split)
print(score_ndarray)
print(score_ndarray.mean())

D:\softWare\anacondas\envs\py365\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
D:\softWare\anacondas\envs\py365\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

[0.79416667 0.77979167 0.80270833 0.79333333 0.79520833]
0.7930416666666668


D:\softWare\anacondas\envs\py365\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [41]:
# 测试集embedding
def getVectorMatrix(content_series):
    vector_list = []
    for content in content_series:
        vector = get_contentVector(jieba.cut(content, True), word2vec_model)
        vector_list.append(vector)
    featureMatrix = np.array(vector_list)
    return featureMatrix
test_df.columns = ['分类', '内容']
startTime = time.time()
featureMatrix = getVectorMatrix(test_df['内容'])
usedTime = time.time() - startTime
print('测试集文本内容向量化花费时间%.2f秒' %usedTime)

测试集文本内容向量化花费时间177.75秒


In [42]:
# 测试集文本内容向量化花费时间177.51秒

In [45]:
# 混淆矩阵
from sklearn.metrics import confusion_matrix

In [46]:
test_label_list = labelEncoder.transform(test_df['分类'])
predict_label_list = logisticRegression_model.predict(featureMatrix)
pd.DataFrame(confusion_matrix(test_label_list, predict_label_list), 
             columns=labelEncoder.classes_,
             index=labelEncoder.classes_ )

体育   健康   女人   娱乐  房地产   教育   文化   新闻   旅游   汽车   科技   财经
体育   945    0    7    8    0    2   15    7   14    0    1    1
健康     0  803   75    0    2   13    6   42    6    1   14   38
女人     6   33  776   44    4    6   75   18   15    9   12    2
娱乐     4    1   51  768    0    5  148   11    5    0    7    0
房地产    3    0   10    1  887    0    6   26   11    3    4   49
教育     0   17   18    4    0  874   18   50    3    2   12    2
文化     6    4   63  182   14   13  599   55   29    8   25    2
新闻    13   31   19   14   27   46   59  605   29    9   44  104
旅游     4    7   38   11   22    3   44   24  818    6   12   11
汽车     6    4    4    0    2    4    4    6   12  917   10   31
科技     2   14   10   11    4   14   30   42   12    5  810   46
财经     1   15   23    1   52    5    6  103   16   24   64  690

In [47]:
# precision_recall_fscore_support固定计算准确率的
from sklearn.metrics import precision_recall_fscore_support

In [48]:
def eval_model(test_label_list, predict_label_list, className_list):
    # 计算每个分类的Precision, Recall, f1, support
    p, r, f1, s = precision_recall_fscore_support(test_label_list, predict_label_list)
    # 计算总体的平均Precision, Recall, f1, support
    total_p = np.average(p, weights=s)
    total_r = np.average(r, weights=s)
    total_f1 = np.average(f1, weights=s)
    total_s = np.sum(s)
    res1 = pd.DataFrame({
        u'Label': className_list,
        u'Precision': p,
        u'Recall': r,
        u'F1': f1,
        u'Support': s
    })
    res2 = pd.DataFrame({
        u'Label': ['总体'],
        u'Precision': [total_p],
        u'Recall': [total_r],
        u'F1': [total_f1],
        u'Support': [total_s]
    })
    res2.index = [999]
    res = pd.concat([res1, res2])
    return res[['Label', 'Precision', 'Recall', 'F1', 'Support']]

eval_model(test_label_list, predict_label_list, labelEncoder.classes_)

Label  Precision  Recall        F1  Support
0      体育   0.954545   0.945  0.949749     1000
1      健康   0.864370   0.803  0.832556     1000
2      女人   0.709324   0.776  0.741165     1000
3      娱乐   0.735632   0.768  0.751468     1000
4     房地产   0.874753   0.887  0.880834     1000
5      教育   0.887310   0.874  0.880605     1000
6      文化   0.593069   0.599  0.596020     1000
7      新闻   0.611729   0.605  0.608346     1000
8      旅游   0.843299   0.818  0.830457     1000
9      汽车   0.931911   0.917  0.924395     1000
10     科技   0.798030   0.810  0.803970     1000
11     财经   0.706967   0.690  0.698381     1000
999    总体   0.792578   0.791  0.791495    12000

In [49]:
# 刚刚做的文本分类预测是属于多标签预测还是多分类预测呢？
    # 12大类，我们每个样本都要做一个12分类
    # 多分类，总体样本，可以有多个分类3+，但是每条样本只有一个分类【1对1】。
    # 多标签，总体样本，也可以有多个分类3+，但是每条样本可能对应多个分类【1对多】。

In [2]:
# FastText
# FastText是facebook开源的一个词向量与文本分类工具
# 是word2vec作者Mikolov转战 Facebook 后于2016年7月发表在论文Bag of Tricks for Efficient Text Classification上。

![jupyter](images/weeks3_img16.png)

In [ ]:
# 可以看到，和 CBOW 一样，
    # fastText 模型也只有三层：输入层、隐含层、输出层（Hierarchical Softmax），
    # 输入都是多个经向量表示的单词，
    # 输出都是一个特定的target，
    # 隐含层都是对多个词向量的叠加平均。
# 不同的是
    # CBOW 的输入是目标单词的上下文，fastText 的输入是多个单词及其 n-gram 特征，这些特征用来表示单个文档；
    # CBOW 的输出是目标词汇，fastText 的输出是文档对应的类标。

In [ ]:
# 关于n-gram特征
# 我们在分词的时候也有给大家讲过，它是一种基于语言模型的算法。
# 基本思想是将文本内容,按照字节顺序,进行大小为N的滑动窗口操作，最终形成长度为N的字节片段序列。

In [ ]:
# 字粒度 n-gram
# 我来到小豚学习人工智能
# 相应的bigram特征为：我来 来到 到小 小豚 豚学 学习 习人 人工 工智 智能
# 相应的trigram特征为：我来到 来到小 到小豚 小豚学 豚学习 学习人 习人工 人工智 工智能

In [ ]:
# 词粒度 n-gram
# 我 来到 小豚 学习 人工智能
# 相应的bigram特征为：我/来到 来到/小豚 小豚/学习 学习/人工智能
# 相应的trigram特征为：我/来到/小豚 来到/小豚/学习 小豚/学习/人工智能

In [ ]:
# 关于input embedding
# 词embedding + n-gram embedding + subword n-gram（W2V有OOV问题，fasttext为了克服这个问题，引入subword）
# 相比word2vec把语料库中的每个单词当成原子的，它会为每个单词生成一个向量
# 这忽略了单词内部的形态特征，比如：apple 和apples，这两个单词都有较多公共字符。
# 即它们的内部形态类似，但是在传统的word2vec中，这种单词内部形态信息因为它们被转换成不同的 id 丢失了。

In [ ]:
# subword n-gram
# 假设我们希望抽取google所有的tri-gram信息，可以得到如下集合：G = { <go, goo, oog, ogl, gle, le>}。
# 在实践中，我们往往会同时提取单词的多种n-gram信息，如2/3/4/5-gram。
# 这样，原始的一个单词google，就被一个字符级别的n-gram集合所表达。
# apples、apple，清华大学和清华

In [ ]:
# 在训练过程中，每个n-gram都会对应训练一个向量，而原来完整单词的词向量就由它对应的所有 n-gram 的向量求和得到。
# 所有的单词向量以及字符级别的 n-gram 向量会同时相加求平均作为训练模型的输入。

In [ ]:
# 好处：
    # 低频词、罕见词：由于在语料中本身出现的次数就少，往往会得不到足够的训练，效果不佳；
    # 而fastText通过引入subword n-gram information的技巧，对于低频词生成的词向量效果会更好。因为它们的 n-gram 可以和其它词共享。
    
    # 未登录词：如果出现了一些在词典中都没有出现过的词，或者带有某些拼写错误的词，传统模型更加无能为力。
    # 而fastText通过引入subword n-gram information的技巧，对于训练词库之外的单词，仍然可以构建它们的词向量。
    # 我们可以叠加它们的字符级 n-gram 向量。
    
    # 语序：词袋模型不能考虑词之间的顺序，而 fastText 通过引入 subword n-gram information 的技巧,来捕获一定的局部序列次序信息。
    # 例如，“我 爱 她” 这句话中的词袋模型特征是 “我”，“爱”, “她”。这些特征和句子 “她 爱 我” 的特征是一样的。
    # 如果加入字符级别的 2-ngram特征，第一句话的特征还有 “我-爱” 和 “爱-她”，这两句话 “我 爱 她” 和 “她 爱 我” 就能区别开来了。

In [51]:
def word_cut(content, stopword_set):
    cutWords = [k for k in jieba.cut(content, True) if k not in stopword_set]
    return " ".join(cutWords)

In [52]:
# 12W有标注的情感倾向评论
ft_all_data = pd.read_csv("data/weibo_senti_100k.csv")
print("评论数目（总体）：%d" % ft_all_data.shape[0])
print("评论数目（正向）：%d" % ft_all_data[ft_all_data.label==1].shape[0])
print("评论数目（负向）：%d" % ft_all_data[ft_all_data.label==0].shape[0])

评论数目（总体）：119988
评论数目（正向）：59993
评论数目（负向）：59995


In [53]:
# train_test_split，训练样本切分的包
from sklearn.model_selection import train_test_split
ft_train_df, ft_test_df = train_test_split(ft_all_data, test_size=0.3, random_state=123)

In [54]:
fasttext_train_df = ft_train_df.copy()
fasttext_train_df.shape

(83991, 2)

In [55]:
fasttext_train_df.head()

label                                             review
80101       0  这是药材 //@虫之微醺:小时候最怕这个了，到现在看见都魂飞魄散的//@新街口老张: 翻过来...
35114       1  [鼓掌]大家都去过哪些地方呢？[心] 点击百度旅游页面右上角的“个人中心”里的“我的足迹”，...
105535      0  韩国过分了啊！！！ //@方寸星辉:中国媒体像吃了屎一样闭嘴了，又在下一盘大大的棋，日// ...
110381      0                                  明天怎么上班阿，都爬不起来了[泪]
21711       1  [哈哈][哈哈][哈哈][哈哈][哈哈]//@coaching旅人:乐翻了 //@Leah_...

In [56]:
# 训练样本内容分词
fasttext_train_df["review"] = fasttext_train_df["review"].map(lambda x: word_cut(x, stopword_set))
fasttext_train_df.head()

label                                             review
80101       0  药材   //@ 虫 微醺 : 小时 小时候 最怕 现在 看见 魂飞魄散 //@ 新街 新街...
35114       1  [ 鼓掌 ] 去过 哪些地方 地方 ？[ 心 ]   点击 百度 旅游 页面 右上 右上角 ...
105535      0  韩国 过分 ！！！   //@ 方寸 星 辉 : 中国 媒体 吃 屎 闭嘴 一盘 大大 大大...
110381      0                             明天 怎么 上班 爬不起来 起来 [ 泪 ]
21711       1  [ ][ ][ ][ ][ ]//@ coaching 旅人 : 乐 翻   //@ Lea...

In [57]:
# label映射
label_map ={}
for name, group in fasttext_train_df.groupby(fasttext_train_df.columns[0]):
    print(name, len(group))
    label_map[name] = "__label__" + str(name)
label_map

0 42033
1 41958


{0: '__label__0', 1: '__label__1'}

In [58]:
fasttext_train_df["label"] = fasttext_train_df["label"].map(lambda x: label_map[x])

In [59]:
fasttext_train_df.head(5)

label                                             review
80101   __label__0  药材   //@ 虫 微醺 : 小时 小时候 最怕 现在 看见 魂飞魄散 //@ 新街 新街...
35114   __label__1  [ 鼓掌 ] 去过 哪些地方 地方 ？[ 心 ]   点击 百度 旅游 页面 右上 右上角 ...
105535  __label__0  韩国 过分 ！！！   //@ 方寸 星 辉 : 中国 媒体 吃 屎 闭嘴 一盘 大大 大大...
110381  __label__0                             明天 怎么 上班 爬不起来 起来 [ 泪 ]
21711   __label__1  [ ][ ][ ][ ][ ]//@ coaching 旅人 : 乐 翻   //@ Lea...

In [60]:
# 测试集，和训练集执行相同的操作
fasttext_test_df = ft_test_df.copy()
fasttext_test_df.shape

(35997, 2)

In [61]:
# 测试集，内容分词
fasttext_test_df["review"] = fasttext_test_df["review"].map(lambda x: word_cut(x, stopword_set))
fasttext_test_df.head()

label                                             review
95382      0  收 税 倒 干点 纳税 纳税人 痛快 事 ！//@ GG 走 天下 : 真是 流氓 双重 加...
21129      1                                美食 有无 无敌 夜景 ?[ 鼓掌 ]
56307      1                                  发现 发 微 博 滴 [ 可爱 ]
53980      1  [ ] 纯属 纯属巧合 巧合 ! 这位 乔 身材 太棒 太棒了 ![ 赞 ]   //@ 中...
31933      1  德 乐 社 相声 山东 山东快书 快板 真值 值得 静静 静听 听听 [ 围观 ][ 围观 ...

In [62]:
# 测试集，标签映射
fasttext_test_df["label"] = fasttext_test_df["label"].map(lambda x: label_map[x])

In [63]:
fasttext_test_df.head(5)

label                                             review
95382  __label__0  收 税 倒 干点 纳税 纳税人 痛快 事 ！//@ GG 走 天下 : 真是 流氓 双重 加...
21129  __label__1                                美食 有无 无敌 夜景 ?[ 鼓掌 ]
56307  __label__1                                  发现 发 微 博 滴 [ 可爱 ]
53980  __label__1  [ ] 纯属 纯属巧合 巧合 ! 这位 乔 身材 太棒 太棒了 ![ 赞 ]   //@ 中...
31933  __label__1  德 乐 社 相声 山东 山东快书 快板 真值 值得 静静 静听 听听 [ 围观 ][ 围观 ...

In [64]:
# dataframe互换两列的位置，大家可以学一些
test_mid = fasttext_test_df['review']
fasttext_test_df.drop(labels=['review'], axis=1,inplace = True)
fasttext_test_df.insert(0, 'review', test_mid)
fasttext_test_df.head(5)

review       label
95382  收 税 倒 干点 纳税 纳税人 痛快 事 ！//@ GG 走 天下 : 真是 流氓 双重 加...  __label__0
21129                                美食 有无 无敌 夜景 ?[ 鼓掌 ]  __label__1
56307                                  发现 发 微 博 滴 [ 可爱 ]  __label__1
53980  [ ] 纯属 纯属巧合 巧合 ! 这位 乔 身材 太棒 太棒了 ![ 赞 ]   //@ 中...  __label__1
31933  德 乐 社 相声 山东 山东快书 快板 真值 值得 静静 静听 听听 [ 围观 ][ 围观 ...  __label__1

In [65]:
# dataframe互换两列的位置
train_mid = fasttext_train_df['review']
fasttext_train_df.drop(labels=['review'], axis=1,inplace = True)
fasttext_train_df.insert(0, 'review', train_mid)
fasttext_train_df.head(5)

review       label
80101   药材   //@ 虫 微醺 : 小时 小时候 最怕 现在 看见 魂飞魄散 //@ 新街 新街...  __label__0
35114   [ 鼓掌 ] 去过 哪些地方 地方 ？[ 心 ]   点击 百度 旅游 页面 右上 右上角 ...  __label__1
105535  韩国 过分 ！！！   //@ 方寸 星 辉 : 中国 媒体 吃 屎 闭嘴 一盘 大大 大大...  __label__0
110381                             明天 怎么 上班 爬不起来 起来 [ 泪 ]  __label__0
21711   [ ][ ][ ][ ][ ]//@ coaching 旅人 : 乐 翻   //@ Lea...  __label__1

In [66]:
fasttext_test_df.to_csv("data/ft_test_df.txt", sep="\t", index=False, header=None)

In [67]:
fasttext_train_df.to_csv("data/ft_train_df.txt", sep="\t", index=False, header=None)

In [69]:
import fasttext
"""
  训练一个监督模型, 返回一个模型对象

  @param input:           训练数据文件路径
  @param lr:              学习率
  @param dim:             向量维度
  @param ws:              cbow模型时使用
  @param epoch:           次数
  @param minCount:        词频阈值, 小于该值在初始化时会过滤掉
  @param minCountLabel:   类别阈值，类别小于该值初始化时会过滤掉
  @param minn:            构造subword时最小char个数
  @param maxn:            构造subword时最大char个数
  @param neg:             负采样
  @param wordNgrams:      n-gram个数
  @param loss:            损失函数类型, softmax, ns: 负采样, hs: 分层softmax
  @param bucket:          词扩充大小, [A, B]: A语料中包含的词向量, B不在语料中的词向量
  @param thread:          线程个数, 每个线程处理输入数据的一段, 0号线程负责loss输出
  @param lrUpdateRate:    学习率更新
  @param t:               负采样阈值
  @param label:           类别前缀
  @param pretrainedVectors: 预训练的词向量文件路径, 如果word出现在文件夹中初始化不再随机
  @return model object
"""
# label的名字，和面前定义的前缀要对应上
classifier = fasttext.train_supervised(input='data/ft_train_df.txt', dim=100, epoch=10,
                                         lr=0.1, wordNgrams=2, loss='softmax', label="__label__")

In [70]:
classifier.save_model('models/ft_classifier.model')

In [71]:
result = classifier.test('data/ft_test_df.txt')
print(result)

(35997, 0.9655249048531822, 0.9655249048531822)


In [72]:
# 别忘了括号
classifier.predict(["奥运 明星 明星写真 写真 写真集 集锦 曝光 　 展现 健康 时尚 图 来源 人民"])

([['__label__1']], [array([0.99953485], dtype=float32)])

In [73]:
import numpy as np
fasttext_test_df_ = np.array(fasttext_test_df["review"]).tolist()

In [74]:
fasttext_test_df_[:2]

['收 税 倒 干点 纳税 纳税人 痛快 事 ！//@ GG 走 天下 : 真是 流氓 双重 加税 过路 过路费 路费 没 解决 变相 税种 //@ 吃饭 睡觉 上班 :   吃饭 喝水 缴税   ！[ 衰 ]//@ edisontao :   不能 有点 有点儿 点儿 创造 创造力 挣钱 不能 低级',
 '美食 有无 无敌 夜景 ?[ 鼓掌 ]']

In [75]:
fasttext_test_df.head()

review       label
95382  收 税 倒 干点 纳税 纳税人 痛快 事 ！//@ GG 走 天下 : 真是 流氓 双重 加...  __label__0
21129                                美食 有无 无敌 夜景 ?[ 鼓掌 ]  __label__1
56307                                  发现 发 微 博 滴 [ 可爱 ]  __label__1
53980  [ ] 纯属 纯属巧合 巧合 ! 这位 乔 身材 太棒 太棒了 ![ 赞 ]   //@ 中...  __label__1
31933  德 乐 社 相声 山东 山东快书 快板 真值 值得 静静 静听 听听 [ 围观 ][ 围观 ...  __label__1

In [76]:
z = labelEncoder.fit_transform(fasttext_train_df['label'])

In [77]:
from sklearn.metrics import confusion_matrix
test_label_list = labelEncoder.transform(fasttext_test_df['label'])
test_label_list[:5]

array([0, 1, 1, 1, 1])

In [78]:
predict_label_list = [int(y.strip("__label__")) for x in fasttext_test_df_ for y in classifier.predict(x)[0]]
predict_label_list[:5]

[0, 1, 1, 1, 1]

In [79]:
pd.DataFrame(confusion_matrix(test_label_list, predict_label_list), 
             columns=labelEncoder.classes_,
             index=labelEncoder.classes_ )

__label__0  __label__1
__label__0       17295         667
__label__1         570       17465

In [80]:
def ft_eval_model(test_label_list, predict_label_list, className_list):
    # 计算每个分类的Precision, Recall, f1, support
    p, r, f1, s = precision_recall_fscore_support(test_label_list, predict_label_list)
    # 计算总体的平均Precision, Recall, f1, support
    total_p = np.average(p, weights=s)
    total_r = np.average(r, weights=s)
    total_f1 = np.average(f1, weights=s)
    total_s = np.sum(s)
    res1 = pd.DataFrame({
        u'Label': className_list,
        u'Precision': p,
        u'Recall': r,
        u'F1': f1,
        u'Support': s
    })
    res2 = pd.DataFrame({
        u'Label': ['总体'],
        u'Precision': [total_p],
        u'Recall': [total_r],
        u'F1': [total_f1],
        u'Support': [total_s]
    })
    res2.index = [999]
    res = pd.concat([res1, res2])
    return res[['Label', 'Precision', 'Recall', 'F1', 'Support']]

ft_eval_model(test_label_list, predict_label_list, labelEncoder.classes_)

Label  Precision    Recall        F1  Support
0    __label__0   0.968094  0.962866  0.965473    17962
1    __label__1   0.963214  0.968395  0.965798    18035
999          总体   0.965649  0.965636  0.965636    35997

In [ ]:
# 作业1，fasttext刚才做的是二分类，大家下去基于word2vec + lr的数据集，尝试下12分类。